In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import geocoder

In [2]:
citys = ['Windsor', 'London', 'Hamilton', 'Burlington', 'Toronto', 'Pickering', 'Ajax','Whitby', 'Oshawa', 'Bowmanville', 'Cobourg', 'Belleville', 'Kingston']

urls = []
price = [] 
building_type = []
rooms = []
address = []
location = []
latitude = []
longitude = []

for city in citys:
    urls.append("https://www.royallepage.ca/en/on/{0}/properties/".format(city))
    
    headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

for url in urls:


    n_pages = 0

    for page in range(1,23):
        n_pages += 1
        url_full = url + str(page)
        response_cobourg = get(url, headers=headers)
        page_html = BeautifulSoup(response_cobourg.text, 'html.parser')
        house_containers = page_html.find_all('div', class_="card card--listing-card js-listing js-property-details")
       
        for containers in house_containers: 
            building = containers.find_all('span')[2].text 
            if building == 'House' or building == 'Vacant Land' or building == 'Townhouse' or building == 'Condo': 
                building_type.append(building) 
                if building == 'new listing':
                    building_type.append(containers.find_all('span')[1])
            else: building_type.append(containers.find_all('span')[0].text) 
            if building == 'House' or building == 'Vacant Land' or building == 'Townhouse' or building == 'Condo': 
                price.append(containers.find_all('span')[4].text)
            else: price.append(containers.find_all('span')[2].text)
            if building == 'House' or building == 'Vacant Land' or building == 'Townhouse' or building == 'Condo':
                rooms.append(containers.find_all('span')[3].text) 
            else: rooms.append(containers.find_all('span')[1].text)
            if building == 'House' or building == 'Vacant Land' or building == 'Townhouse' or building == 'Condo':
                address.append(containers.find_all('a')[1].text) 
            else: address.append(containers.find_all('a')[1].text)
            location.append(url.rsplit('/', 3)[-3])
            latitude.append(' '.join([str(elem) for elem in list(containers.find_all("span", class_ ="ll-match-score"))])[58:68])
            longitude.append(' '.join([str(elem) for elem in list(containers.find_all("span", class_ ="ll-match-score"))])[81:91])

In [4]:
price

['$649,900',
 '$500,000',
 '$499,900',
 '$499,900',
 '$479,900',
 'Commercial',
 '$379,900',
 '$299,999',
 '$219,900',
 '$179,900',
 '$159,900',
 '$599,900',
 '$599,900',
 '$575,000',
 '$499,900',
 '$439,900',
 '$399,900',
 '$399,900',
 '$379,900',
 '$359,900',
 '$329,900',
 '$319,900',
 '$299,900',
 '$299,900',
 '$299,900',
 '$289,000',
 '$249,900',
 '$239,900',
 '$238,000',
 '$225,000',
 '$199,900',
 '$599,000',
 '$349,900',
 '$299,900',
 '$249,900',
 '$199,999',
 '$139,900',
 '$139,900',
 '$349,900',
 '$249,000',
 '$399,500',
 '$649,900',
 '$500,000',
 '$499,900',
 '$499,900',
 '$479,900',
 'Commercial',
 '$379,900',
 '$299,999',
 '$219,900',
 '$179,900',
 '$159,900',
 '$599,900',
 '$599,900',
 '$575,000',
 '$499,900',
 '$439,900',
 '$399,900',
 '$399,900',
 '$379,900',
 '$359,900',
 '$329,900',
 '$319,900',
 '$299,900',
 '$299,900',
 '$299,900',
 '$289,000',
 '$249,900',
 '$239,900',
 '$238,000',
 '$225,000',
 '$199,900',
 '$599,000',
 '$349,900',
 '$299,900',
 '$249,900',
 '$199,9

In [7]:
data = [price, building_type, rooms, address, location, latitude, longitude]
df = pd.concat([pd.Series(x) for x in data], axis=1)

In [8]:
df

,0,1,2,3,4,5,6
0,"$649,900",House,"6 bds , 3 bths",412 Church St,Windsor,42.3152526,83.0421161
1,"$500,000",House,"3 bds , 3 bths",1143 Hickory,Windsor,42.3189102,82.9997162
2,"$499,900",House,"5 bds , 3 bths",3196 Avondale Avenue,Windsor,42.2677498,83.0138996
3,"$499,900",Condo,"2 bds , 3 bths",5955 Ontario Unit# 105,Windsor,42.3271511,82.9645638
4,"$479,900",House,"3 bds , 2 bths",1054 Ivanhill,Windsor,42.3258351,82.9145532
...,...,...,...,...,...,...,...
12953,"$1,095,000",House,"3 bds , 2 bths",91 Gore St,Kingston,44.2271809,"76.485672"""
12954,"$624,900",House,"4 bds , 3 bths",268 Holden Street,Kingston,44.2656851,-76.589800
12955,"$599,900",House,"5 bds , 3 bths",778 Brock St,Kingston,44.2321918,76.5118144
12956,"$524,900",Townhouse,"3 bds , 3 bths",1 Place D'Armes # 52,Kingston,"44.234139""","6.479333"">"


In [9]:
df.rename(columns={0: 'Price'}, inplace=True)
df.rename(columns={1: 'Building_Type'}, inplace=True)
df.rename(columns={2: 'Rooms'}, inplace=True)
df.rename(columns={3: 'Address'}, inplace=True)
df.rename(columns={4: 'City'}, inplace=True)
df.rename(columns={5: 'Latitude'}, inplace=True)
df.rename(columns={6: 'Longitude'}, inplace=True)

In [10]:
df

,Price,Building_Type,Rooms,Address,City,Latitude,Longitude
0,"$649,900",House,"6 bds , 3 bths",412 Church St,Windsor,42.3152526,83.0421161
1,"$500,000",House,"3 bds , 3 bths",1143 Hickory,Windsor,42.3189102,82.9997162
2,"$499,900",House,"5 bds , 3 bths",3196 Avondale Avenue,Windsor,42.2677498,83.0138996
3,"$499,900",Condo,"2 bds , 3 bths",5955 Ontario Unit# 105,Windsor,42.3271511,82.9645638
4,"$479,900",House,"3 bds , 2 bths",1054 Ivanhill,Windsor,42.3258351,82.9145532
...,...,...,...,...,...,...,...
12953,"$1,095,000",House,"3 bds , 2 bths",91 Gore St,Kingston,44.2271809,"76.485672"""
12954,"$624,900",House,"4 bds , 3 bths",268 Holden Street,Kingston,44.2656851,-76.589800
12955,"$599,900",House,"5 bds , 3 bths",778 Brock St,Kingston,44.2321918,76.5118144
12956,"$524,900",Townhouse,"3 bds , 3 bths",1 Place D'Armes # 52,Kingston,"44.234139""","6.479333"">"


In [11]:
df.to_csv(r'C:\Users\gbier\Documents\Real_Estate\_All_Real_Estate_Locations.csv')